<a href="https://colab.research.google.com/github/Iamkoss/cis9650_flight_data/blob/main/fligh_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Flight Data from the Buffalo, NY Airport

### Author: Anna Bold, Kossi Gamli, Ran Xin Jiang, Christina Pratas
### Date : December 8th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction
This projects examines flight records from Buffalo Niagara International Airport (BUF) for the entirety of January 2025. The following 30 JSON files contain detailed information regarding flight schedules, delays, airports, and timezones, etc. The dataset shows how flights operated throughout the month, including patterns in delays, cancellations, airline performance, and schedule consistency. By organizing this raw data into a structured format, it highlights these trends and provides a clearer picture of BUF's activity during this period.

## Problem Statement / Research Question

The research question is finding out what overlying issues the Buffalo Niagara International Airport may be having with the daily arriving and departing flights in January 2025. There are several different factors to look at, including potential problematic airlines, large discrepancies between scheduled departure/arrival times and actual departure/arrival times, and outlying flight cancellations.


As Buffalo in January is subject to inclement weather, research will also look into if any singular days are outliers in regards to flight delays and cancellations due to weather.

## Data Description

## Setup and Environment

In [ ]:
!pip install google-cloud-storage

In [ ]:
!git clone https://github.com/Iamkoss/cis9650_flight_data.git

fatal: destination path 'cis9650_flight_data' already exists and is not an empty directory.


In [ ]:
from google.cloud import storage
from io import BytesIO
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
import glob
from google.cloud import storage
from io import BytesIO

## Data Loading

In [ ]:
# Find all BUF json files
files = sorted(glob.glob("/content/cis9650_flight_data/BUF*.json"))

# Store each file's data as a *separate* DataFrame
flight_dfs = {}  # dictionary: file_name → DataFrame

for file in files:
    print("Loading:", (file))

    with open(file, "r") as f:
        data = json.load(f)

    df = pd.json_normalize(data)

    # Save each DataFrame with name like "BUF_1"
    key = os.path.basename(file).replace(".json", "")
    flight_dfs[key] = df

print("\nLoaded files:", list(flight_dfs.keys()))

Loading: /content/cis9650_flight_data/BUF_1.json
Loading: /content/cis9650_flight_data/BUF_10.json
Loading: /content/cis9650_flight_data/BUF_11.json
Loading: /content/cis9650_flight_data/BUF_12.json
Loading: /content/cis9650_flight_data/BUF_13.json
Loading: /content/cis9650_flight_data/BUF_14.json
Loading: /content/cis9650_flight_data/BUF_15.json
Loading: /content/cis9650_flight_data/BUF_16.json
Loading: /content/cis9650_flight_data/BUF_17.json
Loading: /content/cis9650_flight_data/BUF_18.json
Loading: /content/cis9650_flight_data/BUF_19.json
Loading: /content/cis9650_flight_data/BUF_2.json
Loading: /content/cis9650_flight_data/BUF_20.json
Loading: /content/cis9650_flight_data/BUF_21.json
Loading: /content/cis9650_flight_data/BUF_22.json
Loading: /content/cis9650_flight_data/BUF_23.json
Loading: /content/cis9650_flight_data/BUF_24.json
Loading: /content/cis9650_flight_data/BUF_25.json
Loading: /content/cis9650_flight_data/BUF_26.json
Loading: /content/cis9650_flight_data/BUF_27.json
Lo

In [ ]:
# Empty list to store all flight data
all_flight_data = []

# The key is the filenames
# df is the DataFrame
for key, df in flight_dfs.items():
  records = df.to_dict(orient='records')
  all_flight_data.extend(records)

print("Total flights:", len(all_flight_data))
print("Example record:", all_flight_data[0])

Total flights: 5509
Example record: {'flight_date': '2025-01-05', 'flight_status': 'landed', 'live': None, 'departure.airport': 'Buffalo Niagara International', 'departure.timezone': 'America/New_York', 'departure.iata': 'BUF', 'departure.icao': 'KBUF', 'departure.terminal': None, 'departure.gate': '15', 'departure.delay': 20.0, 'departure.scheduled': '2025-01-05T05:00:00+00:00', 'departure.estimated': '2025-01-05T05:00:00+00:00', 'departure.actual': '2025-01-05T05:20:00+00:00', 'departure.estimated_runway': '2025-01-05T05:20:00+00:00', 'departure.actual_runway': '2025-01-05T05:20:00+00:00', 'arrival.airport': 'Orlando International', 'arrival.timezone': 'America/New_York', 'arrival.iata': 'MCO', 'arrival.icao': 'KMCO', 'arrival.terminal': 'A', 'arrival.gate': '129', 'arrival.baggage': '13', 'arrival.scheduled': '2025-01-05T07:50:00+00:00', 'arrival.delay': nan, 'arrival.estimated': '2025-01-05T07:32:00+00:00', 'arrival.actual': '2025-01-05T07:33:00+00:00', 'arrival.estimated_runway': 

In [18]:
flight_record = df.to_dict(orient="records")

df_all = pd.json_normalize(flight_record)
display(df_all.head())
print(df_all.shape)
print(len(df_all))

,flight_date,flight_status,aircraft,live,departure.airport,departure.timezone,departure.iata,departure.icao,departure.terminal,departure.gate,...,flight.codeshared.airline_iata,flight.codeshared.airline_icao,flight.codeshared.flight_number,flight.codeshared.flight_iata,flight.codeshared.flight_icao,flight.codeshared,aircraft.registration,aircraft.iata,aircraft.icao,aircraft.icao24
0,2025-01-28,landed,NaN,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,4,...,aa,aal,5312,aa5312,aal5312,NaN,NaN,NaN,NaN,NaN
1,2025-01-28,landed,NaN,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,4,...,NaN,NaN,NaN,NaN,NaN,NaN,N516AE,CRJ7,CRJ7,A677BE
2,2025-01-28,landed,NaN,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,5,...,NaN,NaN,NaN,NaN,NaN,NaN,N129AA,A321,A321,A07522
3,2025-01-28,landed,NaN,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,2A,...,NaN,NaN,NaN,NaN,NaN,NaN,N652RS,E145,E145,A895B0
4,2025-01-28,landed,NaN,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,8,...,b6,jbu,141,b6141,jbu141,NaN,NaN,NaN,NaN,NaN


(173, 46)
173


In [ ]:
print(files)
print(len(files))


['/content/cis9650_flight_data/BUF_1.json', '/content/cis9650_flight_data/BUF_10.json', '/content/cis9650_flight_data/BUF_11.json', '/content/cis9650_flight_data/BUF_12.json', '/content/cis9650_flight_data/BUF_13.json', '/content/cis9650_flight_data/BUF_14.json', '/content/cis9650_flight_data/BUF_15.json', '/content/cis9650_flight_data/BUF_16.json', '/content/cis9650_flight_data/BUF_17.json', '/content/cis9650_flight_data/BUF_18.json', '/content/cis9650_flight_data/BUF_19.json', '/content/cis9650_flight_data/BUF_2.json', '/content/cis9650_flight_data/BUF_20.json', '/content/cis9650_flight_data/BUF_21.json', '/content/cis9650_flight_data/BUF_22.json', '/content/cis9650_flight_data/BUF_23.json', '/content/cis9650_flight_data/BUF_24.json', '/content/cis9650_flight_data/BUF_25.json', '/content/cis9650_flight_data/BUF_26.json', '/content/cis9650_flight_data/BUF_27.json', '/content/cis9650_flight_data/BUF_28.json', '/content/cis9650_flight_data/BUF_29.json', '/content/cis9650_flight_data/BUF

In [ ]:
#Convert the pandas DataFrame `df_all` into a list of dictionaries where each row represents a flight record from the DataFrame
# The `orient="records"` argument ensures that each row is converted into a dictionary where column names and cell values are corresponding

flight_record = df_all.to_dict(orient="records")

# Print the first dictionary (flight record) in the `flight_record` list.
# This provides a quick way to inspect the structure and content of a single flight entry.
print(flight_record[0])

# Print the total number of flight records loaded into the `flight_record` list.
# This indicates the total count of individual flights being analyzed.
print("Total flights:", len(flight_record))

{'flight_date': '2025-01-28', 'flight_status': 'landed', 'aircraft': nan, 'live': None, 'departure.airport': 'Buffalo Niagara International', 'departure.timezone': 'America/New_York', 'departure.iata': 'BUF', 'departure.icao': 'KBUF', 'departure.terminal': None, 'departure.gate': '4', 'departure.delay': 22.0, 'departure.scheduled': '2025-01-28T05:21:00+00:00', 'departure.estimated': '2025-01-28T05:21:00+00:00', 'departure.actual': '2025-01-28T05:42:00+00:00', 'departure.estimated_runway': '2025-01-28T05:42:00+00:00', 'departure.actual_runway': '2025-01-28T05:42:00+00:00', 'arrival.airport': 'Ronald Reagan Washington National Airport', 'arrival.timezone': 'America/New_York', 'arrival.iata': 'DCA', 'arrival.icao': 'KDCA', 'arrival.terminal': '2', 'arrival.gate': 'E55', 'arrival.baggage': '10', 'arrival.scheduled': '2025-01-28T06:52:00+00:00', 'arrival.delay': nan, 'arrival.estimated': None, 'arrival.actual': None, 'arrival.estimated_runway': None, 'arrival.actual_runway': None, 'airline.

In [ ]:
# Calculating the number and percentage of all cancelled flights.
def count_cancelled_flights(df):
    cancelled_flights = df[df['flight_status'] == 'cancelled']
    return len(cancelled_flights)

# Use the function on df_flights to calcuate the number as well as the percentage of all flights that were cancelled.
num_cancelled_flights = count_cancelled_flights(df_flights)
print(f"Number of cancelled flights: {num_cancelled_flights}")
percentage_cancelled_flights = (num_cancelled_flights / len(df_flights)) * 100
print(f"Percentage of cancelled flights: {percentage_cancelled_flights:.2f}%")

Number of cancelled flights: 53
Percentage of cancelled flights: 0.96%


In [ ]:
#creating a table to visualize flight information for all cancelled flights
df_flights = pd.DataFrame(all_flight_data)

cancelled_flights_df = df_flights[df_flights['flight_status'] == 'cancelled']

print(f"Total cancelled flights: {len(cancelled_flights_df)}")
display(cancelled_flights_df.head())

Total cancelled flights: 53


,flight_date,flight_status,live,departure.airport,departure.timezone,departure.iata,departure.icao,departure.terminal,departure.gate,departure.delay,...,aircraft.iata,aircraft.icao,aircraft.icao24,aircraft,flight.codeshared.airline_name,flight.codeshared.airline_iata,flight.codeshared.airline_icao,flight.codeshared.flight_number,flight.codeshared.flight_iata,flight.codeshared.flight_icao
585,2025-01-11,cancelled,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,5,NaN,...,A321,A321,A10BBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,2025-01-11,cancelled,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,4,NaN,...,CRJ7,CRJ7,A97D63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,2025-01-11,cancelled,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,2025-01-06,cancelled,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,3,153.0,...,NaN,NaN,NaN,NaN,frontier airlines,f9,fft,1691,f91691,fft1691
1410,2025-01-06,cancelled,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,3,153.0,...,A20N,A20N,A44AA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def count_landed_delayed_flights(df):
    landed_delayed = df[(df['flight_status'] == 'landed') & (df['departure.delay'] > 0)]
    return len(landed_delayed)

# Use the function on df_flights to calcuate the number as well as the percentage of all flights that were delayed.
num_landed_delayed_flights = count_landed_delayed_flights(df_flights)
print(f"Number of landed flights with a departure delay: {num_landed_delayed_flights}")
percentage_delayed_flights = (num_landed_delayed_flights / total_flights) * 100
print(f"Percentage of landed flights with a departure delay: {percentage_delayed_flights:.2f}%")

Number of landed flights with a departure delay: 4788
Percentage of landed flights with a departure delay: 86.91%


In [ ]:
landed_delayed_flights_df = df_flights[(df_flights['flight_status'] == 'landed') & (df_flights['departure.delay'] > 0)]
display(landed_delayed_flights_df.head())
print(f"Total cancelled flights: {len(landed_delayed_flights_df)}")

,flight_date,flight_status,live,departure.airport,departure.timezone,departure.iata,departure.icao,departure.terminal,departure.gate,departure.delay,...,aircraft.iata,aircraft.icao,aircraft.icao24,aircraft,flight.codeshared.airline_name,flight.codeshared.airline_iata,flight.codeshared.airline_icao,flight.codeshared.flight_number,flight.codeshared.flight_iata,flight.codeshared.flight_icao
0,2025-01-05,landed,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,15,20.0,...,B38M,B38M,AC2C10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-05,landed,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,None,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-05,landed,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,11,38.0,...,NaN,NaN,NaN,NaN,american airlines,aa,aal,5163,aa5163,aal5163
3,2025-01-05,landed,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,11,38.0,...,CRJ7,CRJ7,A6BF4E,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-05,landed,None,Buffalo Niagara International,America/New_York,BUF,KBUF,None,16,4.0,...,B38M,B38M,AC4CA8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Total cancelled flights: 4788


In [ ]:
#NEEDS WORKS, DONT KNOW IF WE WILL USE THIS, IT CAME FROM THE EXAMPLE
def get_file_to_dataframe(bucket_name: str, blob_path: str):



    # Download file content as bytes


    # Detect file extension


    # Convert bytes → Python object

        # If JSON is a list → NORMAL DataFrame

        # If JSON is a dict → flatten into rows
        elif isinstance(data, dict):
            df = pd.json_normalize(data)
        else:
            raise ValueError("Unsupported JSON structure")
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    return df

SyntaxError: incomplete input (ipython-input-4241403107.py, line 15)

In [ ]:
#NEEDS WORKS, BUT IT WILL ALLOW US TO CONVERT THE DICTIONARY INTO COLUMNS, AND THEN WE SHOULD BE ABLE TO SELECT THE COLUMNS WE WANT TO WORK ON.

df_expanded = df.explode("details", ignore_index=True)

# 2. Convert the dict into columns
detail_cols = df_expanded["rows"].apply(pd.Series)

# 3. Merge back and drop the dict column
df_flat = pd.concat([df_expanded.drop(columns=["rows"]), detail_cols], axis=1)

df_flat

KeyError: 'details'

## Data Preparation

In [ ]:
display(df)

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live
0,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'AlphaSky', 'iata': 'AS', 'icao': 'JAG'}","{'number': '6276', 'iata': 'AS6276', 'icao': '...",None,NaN
1,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5312', 'iata': 'AA5312', 'icao': '...","{'registration': 'N516AE', 'iata': 'CRJ7', 'ic...",NaN
2,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Charlotte Douglas', 'timezone': '...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '1204', 'iata': 'AA1204', 'icao': '...","{'registration': 'N129AA', 'iata': 'A321', 'ic...",NaN
3,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5769', 'iata': 'AA5769', 'icao': '...","{'registration': 'N652RS', 'iata': 'E145', 'ic...",NaN
4,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'British Airways', 'iata': 'BA', 'ica...","{'number': '8304', 'iata': 'BA8304', 'icao': '...",None,NaN
...,...,...,...,...,...,...,...,...
168,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'Qatar Airways', 'iata': 'QR', 'icao'...","{'number': '3936', 'iata': 'QR3936', 'icao': '...",None,NaN
169,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'UPS Airlines', 'iata': '5X', 'icao':...","{'number': '1131', 'iata': '5X1131', 'icao': '...","{'registration': 'N473UP', 'iata': 'B752', 'ic...",NaN
170,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Fort Lauderdale Executive', 'time...","{'name': 'Private owner', 'iata': None, 'icao'...","{'number': '148', 'iata': None, 'icao': None, ...",None,NaN
171,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Raleigh-durham International Airp...,"{'name': 'NetJets Aviation', 'iata': '1I', 'ic...","{'number': '958', 'iata': '1I958', 'icao': 'EJ...",None,NaN


In [ ]:
def clean_data(df):
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')

    # Remove duplicate rows
    df.drop_duplicates(inplace=True)

    return df
all_dfs = clean_data(all_dfs)

In [ ]:
display(df)

,flight_date,flight_status,departure,arrival,airline,flight,aircraft,live
0,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'AlphaSky', 'iata': 'AS', 'icao': 'JAG'}","{'number': '6276', 'iata': 'AS6276', 'icao': '...",None,NaN
1,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Ronald Reagan Washington National...,"{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5312', 'iata': 'AA5312', 'icao': '...","{'registration': 'N516AE', 'iata': 'CRJ7', 'ic...",NaN
2,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Charlotte Douglas', 'timezone': '...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '1204', 'iata': 'AA1204', 'icao': '...","{'registration': 'N129AA', 'iata': 'A321', 'ic...",NaN
3,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'American Airlines', 'iata': 'AA', 'i...","{'number': '5769', 'iata': 'AA5769', 'icao': '...","{'registration': 'N652RS', 'iata': 'E145', 'ic...",NaN
4,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'British Airways', 'iata': 'BA', 'ica...","{'number': '8304', 'iata': 'BA8304', 'icao': '...",None,NaN
...,...,...,...,...,...,...,...,...
168,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'John F Kennedy International', 't...","{'name': 'Qatar Airways', 'iata': 'QR', 'icao'...","{'number': '3936', 'iata': 'QR3936', 'icao': '...",None,NaN
169,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Philadelphia International', 'tim...","{'name': 'UPS Airlines', 'iata': '5X', 'icao':...","{'number': '1131', 'iata': '5X1131', 'icao': '...","{'registration': 'N473UP', 'iata': 'B752', 'ic...",NaN
170,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...","{'airport': 'Fort Lauderdale Executive', 'time...","{'name': 'Private owner', 'iata': None, 'icao'...","{'number': '148', 'iata': None, 'icao': None, ...",None,NaN
171,2025-01-28,landed,"{'airport': 'Buffalo Niagara International', '...",{'airport': 'Raleigh-durham International Airp...,"{'name': 'NetJets Aviation', 'iata': '1I', 'ic...","{'number': '958', 'iata': '1I958', 'icao': 'EJ...",None,NaN


## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

1. Barber, David. Bayesian Reasoning and Machine Learning. Cambridge University Press, 2012.
2. Aste, Tomaso, Paola Cerchiello, and Roberta Scaramozzino. "Information-Theoretic Causality Detection between Financial and Sentiment Data."Entropy, vol. 24, no. 6, 2022, pp. 1–18. DOI:10.3390/e24060774.
3. Metz, Cade. "Microsoft Puts OpenAI’s Sam Altman in Charge of New Advanced AI Research Team."
The New York Times, 20 Nov. 2023, www.nytimes.com/2023/11/20/technology/openai-microsoft-altman.html

## Appendix